In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams):
    file = open("results/gridsearch_amanda_dynamic_batch.txt","a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading sinthetic dataset
    #datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard]
    datasets = [setup.load4CR, setup.load4CRE_V1, setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard]
    #real datasets
    #datasets = [setup.loadKeystroke] # setup.loadNOAADataset, setup.loadElecData
   
    #datasets = [ setup.loadCSurr]
    batches=100
    poolSize = None
    isBatchMode = True
    #testing grid search
    for i in range(len(datasets)):
        finalScore = 0
        best_grid={}
        dataValues, dataLabels, description = datasets[i](path, sep)
        initialLabeledData = int(0.05*len(dataLabels))
        sizeOfBatch = int((len(dataValues)-initialLabeledData)/batches)
        print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))
        
        tuned_params = [{"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], "sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData]}]
        
        for g in ParameterGrid(tuned_params):
            averageAccuracy=0
            gs = grid_selection_amanda_dynamic.run(**g)
            
            try:
                gs.fit(dataValues, dataLabels)
                averageAccuracy = np.mean(gs.predict())
                if finalScore < averageAccuracy:
                    finalScore = averageAccuracy
                    best_grid = g
            except Exception:
                print("An error occured in ", description, g)
                #raise Exception
            
        print(finalScore)
        print(best_grid)
        
        writeResults(description, finalScore, best_grid)
    
if __name__ == "__main__":
    main()


Four Classes Rotating Separated. Bidimensional.: 100 batches of 1371 instances


KeyboardInterrupt: 